In [2]:
DATADIR = "../../pzaydel/DM3/Data/hw-ad/"
!head -n 10 ../../pzaydel/DM3/Data/hw-ad/train.csv | awk '{split($0,a,";"); print a[1]}'

timestamp
1379278800
1379278800
1379278800
1379278800
1379278800
1379278800
1379278800
1379278800
1379278800


In [5]:
!head ../../pzaydel/DM3/Data/hw-ad/test.csv | awk '{split($0,a,";"); print a[2]}'

label
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [1]:
import numpy as np
COMBINEDIR = 'combine/'
print("Loading train xgb")
train_xgb_features = np.load(COMBINEDIR + 'trainxgbfeat.npy')
print("Loading test xgb")
test_xgb_features = np.load(COMBINEDIR + 'testxgbfeat.npy')
print("Loaded")

Loading train xgb
Loading test xgb
Loaded


In [11]:
test_xgb_features.shape

(20317220, 30)

In [1]:
import pandas as pd
traindata = pd.read_csv('../../pzaydel/DM3/Data/hw-ad/train.csv', sep = ';')
testdata = pd.read_csv('../../pzaydel/DM3/Data/hw-ad/test.csv', sep = ';')

In [2]:
trainmean = traindata.label.mean()
def fill_mean(df, feat, target_set, m):
    mask = df[feat].apply(lambda x: x in target_set)
    df[feat+"_CTR"][mask] = np.nan

In [3]:
from tqdm import tqdm, tqdm_notebook
import numpy as np
for i in tqdm_notebook(range(1, 13)):
    featname = "C{}".format(i)
    a = traindata[[featname, "label"]].groupby(featname)["label"].mean()
    a = a.to_frame().reset_index()
    a.columns = [a.columns[0], a.columns[0] + "_CTR"]
    traindata = traindata.merge(a, left_on=featname, right_on=featname, how='left')
    testdata = testdata.merge(a, left_on=featname, right_on=featname, how='left')
    left_resid = set(traindata[featname].unique().tolist()) - set(testdata[featname].unique().tolist())
    right_resid = set(testdata[featname].unique().tolist()) - set(traindata[featname].unique().tolist()) 
    val1 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / traindata[featname].unique().shape[0]
    val2 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / testdata[featname].unique().shape[0]
    print("{}\t{}\t{}".format(featname, val1, val2))
    fill_mean(traindata, featname, left_resid, trainmean)
    fill_mean(testdata, featname, right_resid, trainmean)
    del a

C1	0.6109284673312078	0.7074115844652252


/home/vbelyaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


C2	0.6864072194021432	0.8068686600808858
C3	0.848735244519393	0.910126582278481
C4	0.7113091541423425	0.8218937246600408
C5	0.9795918367346939	0.9696969696969697
C6	0.9448094612352168	0.9563713753657888
C7	1.0	0.75
C8	0.8758314855875832	0.9316037735849056
C9	0.9405940594059405	0.9405940594059405
C10	0.8738670694864048	0.9315619967793881
C11	1.0	1.0
C12	1.0	1.0



In [4]:
for l in tqdm_notebook(range(1, 3)):
    featname = "l{}".format(l)
    a = traindata[[featname, "label"]].groupby(featname)["label"].mean()
    a = a.to_frame().reset_index()
    a.columns = [a.columns[0], a.columns[0] + "_CTR"]
    traindata = traindata.merge(a, left_on=featname, right_on=featname, how='left')
    testdata = testdata.merge(a, left_on=featname, right_on=featname, how='left')
    left_resid = set(traindata[featname].unique().tolist()) - set(testdata[featname].unique().tolist())
    right_resid = set(testdata[featname].unique().tolist()) - set(traindata[featname].unique().tolist())
    fill_mean(traindata, featname, left_resid, trainmean)
    fill_mean(testdata, featname, right_resid, trainmean) 
    val1 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / traindata[featname].unique().shape[0]
    val2 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / testdata[featname].unique().shape[0]
    print("{}\t{}\t{}".format(featname, val1, val2))
    del a

/home/vbelyaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


l1	0.7520911803551338	0.8783204798628963
l2	0.7325581395348837	0.9130434782608695



In [10]:
traindata["hour"] = (traindata.timestamp.values % (1440 * 60)) // 3600
testdata["hour"] = (testdata.timestamp.values % (1440 * 60)) // 3600

In [ ]:
for l in ["hour"]:
    featname = l
    a = traindata[[featname, "label"]].groupby(featname)["label"].mean()
    a = a.to_frame().reset_index()
    a.columns = [a.columns[0], a.columns[0] + "_CTR"]
    traindata = traindata.merge(a, left_on=featname, right_on=featname, how='left')
    testdata = testdata.merge(a, left_on=featname, right_on=featname, how='left')
    left_resid = set(traindata[featname].unique().tolist()) - set(testdata[featname].unique().tolist())
    right_resid = set(testdata[featname].unique().tolist()) - set(traindata[featname].unique().tolist())
    fill_mean(traindata, featname, left_resid, trainmean)
    fill_mean(testdata, featname, right_resid, trainmean) 
    val1 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / traindata[featname].unique().shape[0]
    val2 = len(set(testdata[featname].unique().tolist()) & set(traindata[featname].unique().tolist())) / testdata[featname].unique().shape[0]
    print("{}\t{}\t{}".format(featname, val1, val2))
    del a

In [4]:
from tqdm import tqdm
with open(DATADIR + "train.csv", "r") as input_file, open("fintrain.vw", "w") as output_file:
    input_file.readline()
    first_tmstmp = 1379278800
    for idx, line in enumerate(tqdm(input_file, total = 29989752)):
        features = line[:-1].split(';')
        curr_line = "{} |i".format(int(features[1])*2-1)
        # Счетчики
        curr_line += " l1:{} l2:{}".format(features[15], features[16])
        curr_line += " |j l1_{} l2_{}".format(features[15], features[16])
        # Категориальные фичи
        curr_line += " |c"
        for i in range(2,5):
            curr_line += " C{}_{}".format(i-1, features[i])
        curr_line += " |d"
        for i in range(5,12):
            curr_line += " C{}_{}".format(i-1, features[i])
        # Категориальные фичи: среднее
        curr_line += " |e"
        curr_line += " C11_{}".format(features[17])
        curr_line += " C12_{}".format(features[18])
        # Группы
        curr_line += " |m"
        curr_line += " {}".format(" ".join(features[12].split(',')))
        curr_line += " |n"
        curr_line += " {}".format(" ".join(features[13].split(',')))
        curr_line += " |k"
        curr_line += " {}".format(" ".join(features[14].split(',')))
        # Обработаем timestamp -> хочется посмотреть час и дату
        timestamp = int(features[0])
        dayseconds = timestamp % (1440 * 60)
        dayhours = dayseconds // 3600
        curr_line += " |t h_{}".format(dayhours)
        curr_line += " |u hr:{}".format(dayhours)
        #curr_line += " |x"
        #for i in range(train_xgb_features.shape[1]):
            #curr_line += " t{}l{}".format(i, train_xgb_features[idx, i])
        output_file.write(curr_line + "\n")
        
        

100%|██████████| 29989752/29989752 [19:25<00:00, 25729.24it/s]


In [ ]:
print("Finished")

In [9]:
import numpy as np
with open("fintrain.vw",'r') as source, open("strain",'w') as train, open("stest",'w') as test:
    c=0
    for line in tqdm(source, total=29989752):
        c+=1
        if c > 20000000:
            test.write(line)
        else:
            train.write(line)

100%|██████████| 29989752/29989752 [12:04<00:00, 41407.02it/s]


In [7]:
from tqdm import tqdm
with open(DATADIR + "test.csv", "r") as input_file, open("fintest.vw", "w") as output_file:
    input_file.readline()
    
    for idx, line in enumerate(tqdm(input_file, total = 20317221)):
        features = line[:-1].split(';')
        curr_line = "{} |i".format(features[1])
        # Счетчики
        curr_line += " l1:{} l2:{}".format(features[15], features[16])
        curr_line += " |j l1_{} l2_{}".format(features[15], features[16])
        # Категориальные фичи
        curr_line += " |c"
        for i in range(2,5):
            curr_line += " C{}_{}".format(i-1, features[i])
        curr_line += " |d"
        for i in range(5,12):
            curr_line += " C{}_{}".format(i-1, features[i])
        # Группы
        # Категориальные фичи: среднее
        curr_line += " |e"
        curr_line += " C11_{}".format(features[17])
        curr_line += " C12_{}".format(features[18])
        curr_line += " |m"
        curr_line += " {}".format(" ".join(features[12].split(',')))
        curr_line += " |n"
        curr_line += " {}".format(" ".join(features[13].split(',')))
        curr_line += " |k"
        curr_line += " {}".format(" ".join(features[14].split(',')))
        # Обработаем timestamp -> хочется посмотреть час
        timestamp = int(features[0])
        dayseconds = timestamp % (1440 * 60)
        dayhours = dayseconds // 3600
        curr_line += " |t h_{}".format(dayhours)
        curr_line += " |u hr:{}".format(dayhours)
        #curr_line += " |x"
        #for i in range(test_xgb_features.shape[1]):
            #curr_line += " t{}l{}".format(i, test_xgb_features[idx, i])
        output_file.write(curr_line + "\n")
        

100%|█████████▉| 20317220/20317221 [15:22<00:00, 22022.17it/s]


In [10]:
from datetime import datetime
from tqdm import tqdm
import gzip
curr_file = "sample " + str(datetime.now()).replace(":","_").replace(" ","")[:-7]
with open("fullpred124", "r") as input_file, open(curr_file, 'w') as output_file:
    output_file.write("Id,Click\n")
    for idx, line in enumerate(tqdm(input_file)):
        output_file.write("{},{}".format(idx+1, line))

20317220it [02:06, 161232.65it/s]


In [ ]:
import gzip
content = "Lots of content here"
with gzip.open('file.txt.gz', 'wb') as f:
    f.write(content)

In [ ]:
import pandas as pd
train_data = pd.read_csv(DATADIR + 'train.csv', sep=';')

In [21]:
timestamp = int("1379279729")
dayseconds = timestamp % (1440 * 60)
dayhours = dayseconds // 3600
dayhours

21

In [31]:
1379278800 % (1440*60)//3600

21